# Определение тональности отзывов на фильмы в базе IMDB

Пример из курса ["Программирование глубоких нейронных сетей на Python"](https://www.asozykin.ru/courses/nnpython).

Чтобы запускать и редактировать код, сохраните копию этого ноутбука себе (File->Save a copy in Drive...). Свою копию вы сможете изменять и запускать.

In [1]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM, SpatialDropout1D
from keras.datasets import imdb

Using TensorFlow backend.


In [0]:
# Максимальное количество слов (по частоте использования)
max_features = 5000
# Максимальная длина рецензии в словах
maxlen = 80

## Загружаем данные

In [3]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

17465344/17464789 [==============================] - 1s 0us/step


## Предварительная обработка рецензий

In [0]:
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

## Создаем нейронную сеть

In [0]:
model = Sequential()
# Слой для векторного представления слов
model.add(Embedding(max_features, 32))
model.add(SpatialDropout1D(0.2))
# Слой долго-краткосрочной памяти
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2)) 
# Полносвязный слой
model.add(Dense(1, activation="sigmoid"))

**Печаем информацию о сети**

In [11]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          160000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, None, 32)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


**Компилируем модель**

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Обучаем модель

In [13]:
model.fit(X_train, y_train, batch_size=64, epochs=7,
          validation_data=(X_test, y_test), verbose=2)

Train on 25000 samples, validate on 25000 samples
Epoch 1/7
 - 113s - loss: 0.5057 - acc: 0.7472 - val_loss: 0.4011 - val_acc: 0.8225
Epoch 2/7
 - 112s - loss: 0.3716 - acc: 0.8421 - val_loss: 0.3954 - val_acc: 0.8224
Epoch 3/7
 - 111s - loss: 0.3366 - acc: 0.8579 - val_loss: 0.3712 - val_acc: 0.8358
Epoch 4/7
 - 112s - loss: 0.3132 - acc: 0.8720 - val_loss: 0.3839 - val_acc: 0.8342
Epoch 5/7
 - 112s - loss: 0.2981 - acc: 0.8766 - val_loss: 0.4523 - val_acc: 0.8247
Epoch 6/7
 - 111s - loss: 0.2850 - acc: 0.8820 - val_loss: 0.3942 - val_acc: 0.8296
Epoch 7/7
 - 111s - loss: 0.2662 - acc: 0.8885 - val_loss: 0.4470 - val_acc: 0.8224


## Оцениваем качество обучения сети

In [9]:
scores = model.evaluate(X_test, y_test,
                        batch_size=64)
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1] * 100))

25000/25000 [==============================] - 25s 1ms/step
Аккуратность на тестовых данных: 82.96%
